<h2>Lesson subject: match areas to each and every address in particular city</h2>

In this introduction we are going to get data for wanted city and assign areas to addresses without any (human-guided) optimization.

It will be a foundation for our later <i>scan</i> variants mentioned in my blog post.

<ol>
    <li>Load code that do the hard job for us but is not essential to explain domain problem we solve with it. It will eventually become our geospatial/smart citizens library.</li>
    <li>City data setup: attributes mapping and local data paths</li>
</ol> 

In [ ]:
#load """..\src\Domain.fsx""" 
#load """..\src\Cities\Toronto.fsx""" 

open Domain

3) As we're using open data enabled by proper city goverment, they deserve and often require the attribution. 

In [ ]:
Toronto.license.info |> display 

Open Government Licence – Toronto
(https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-licence/)
Attribution: Contains information licensed under the Open Government Licence – Toronto

4) Now we can parse the data. Let's also display short summary of what we got

In [ ]:
let addresses , districts, streets, summary = 
    setup Toronto.data Area.Neighbourhoods (Limit.Streets 10) 

summary |> display

data summary (without limits) -> 
 areas: 140, streets: 9521, addressess: 527189 
 addresses per street (median/mean/std-dev): 32.0/55.3711795/104.1107272

Few mappings that will make our computations easier to reason about later on.
We translate our areas collection to indexed equivalent because plain indexes are the only data we really need to do the matching.

In [ ]:
let positions = addresses |> Array.map (fun x -> x.Pos) 

let indexedDistricts = districts |> Array.indexed

We have everything in place to apply the address/area matching. The execution time depends on the addresses or street limit you provided at #4.

In [ ]:

let noHintSearch currentPosition  =
    let find = snd >> isPointWithinFeature currentPosition
    indexedDistricts |> Array.findIndex find

let addressesAreas = positions |> Array.map noHintSearch 

Display few results:

In [ ]:
addressesAreas 
    |> Seq.skip 520 
    |> Seq.take 10 
    |> Seq.mapi (fun i x -> {| AddressIndex = i + 520; MatchedAreaIndex = x |})

and add a few more formal verifications:

In [ ]:

let test2 = 500
let test3 = 5000

let index1 = addressesAreas.[test1]
let district1 = districts.[index1]
district1.Attributes.["AREA_NAME"] |> display //Long Branch (19) ?

let index2 = addressesAreas.[test2]
let district2 = districts.[index2]
district2.Attributes.["AREA_NAME"] |> display //South Parkdale (85) ?

let index3 = addressesAreas.[test3]
let district3 = districts.[index3]
district3.Attributes.["AREA_NAME"] |> display //Briar Hill-Belgravia (108) ?

That is all for now. Our goal is the optimization of execution time, to see it in practice checkout other lessons (notebooks) in this module.